In [1]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

In [3]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
#Se importan las librerias
import pandas as pd
import numpy as np
import ast
import json
import pickle
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, explode, sum,expr, array_contains, split, substring, concat_ws, lit, udf, collect_list, from_unixtime, year, month, dayofmonth
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pyspark.conf import SparkConf


In [5]:
#Aumenta la memoria de la sesión de spark
configuracion = SparkConf().set("spark.executor.memory", "8g").set("spark.driver.memory", "8g")
spark = SparkSession.builder.appName("Dataframes").config(conf=configuracion).getOrCreate()

In [6]:
#Se cargan los archivos de la carpeta metadata-sitios y se compilan en un solo DataFrame

metadata_sitios=[]

for i in range(1,12):


    archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/metadata-sitios/{i}.json")

    archivo = archivo.withColumn("MISC", col("MISC").cast("string"))

    metadata_sitios.append(archivo)

df_final = metadata_sitios[0]

for dataframe in metadata_sitios[1:]:

  df_final = df_final.unionByName(dataframe)


metadata_sitios=df_final

In [7]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['MISC', 'state', 'price', 'hours', 'description','url']

# Elimina las columnas especificadas
metadata_sitios = metadata_sitios.select([columna for columna in metadata_sitios.columns if columna not in columnas_a_eliminar])

#Pasa la columna 'category' a tipo string
metadata_sitios = metadata_sitios.withColumn("category", col("category").cast("string"))

# Filtra las filas que contienen 'restaurant' en la columna 'category'
metadata_sitios = metadata_sitios.filter((col("category").contains('restaurant')) | (col("category").contains('Restaurant')))

# Utiliza la función substring para eliminar el primer y último caracter de la columna 'category'
metadata_sitios = metadata_sitios.withColumn("category", expr("substring(category, 2, length(category)-2)"))

# Separa la columna 'address' en 3 columnas: 'address', 'city' y 'state'

split_col = split(metadata_sitios['address'], ',')
metadata_sitios = metadata_sitios.withColumn('state', substring(split_col.getItem(3), 2, 2))
metadata_sitios = metadata_sitios.withColumn('city', split_col.getItem(2))
metadata_sitios = metadata_sitios.withColumn('address', concat_ws(', ', split_col.getItem(0), split_col.getItem(1)))

# Filtrar las filas con estados 'FL', 'VA' o 'NV'
estados_seleccionados = ['FL', 'VA', 'NV']
metadata_sitios = metadata_sitios.filter(metadata_sitios['state'].isin(estados_seleccionados))



In [8]:
#Se leen los reviews de Nevada

reviews_Nevada=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Nevada/{i}.json")



        reviews_Nevada.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Nevada[0]

for dataframe in reviews_Nevada[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Nevada=df_final



In [9]:
#Se leen los reviews de Florida

reviews_Florida=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Florida/{i}.json")



        reviews_Florida.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Florida[0]

for dataframe in reviews_Florida[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Florida=df_final

In [10]:
#Se leen los reviews de Virginia

reviews_Virginia=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Virginia/{i}.json")



        reviews_Virginia.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Virginia[0]

for dataframe in reviews_Virginia[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Virginia=df_final

In [11]:
# Agregar la columna 'state' con valores constantes a cada DataFrame
reviews_Nevada = reviews_Nevada.withColumn('state', lit('NV'))
reviews_Florida = reviews_Florida.withColumn('state', lit('FL'))
reviews_Virginia = reviews_Virginia.withColumn('state', lit('VA'))

# Concatenar los DataFrames
reviews_google = reviews_Nevada.union(reviews_Florida).union(reviews_Virginia)

#Transformo la columna 'time' desde el formato Unix

reviews_google = reviews_google.withColumn("time", from_unixtime(col("time") / 1000).cast("timestamp"))

# Crea columnas 'year', 'month' y 'day'
reviews_google = reviews_google.withColumn("year", year("time"))
reviews_google = reviews_google.withColumn("month", month("time"))
reviews_google = reviews_google.withColumn("day", dayofmonth("time"))

# Elimina la columna 'time'
reviews_google = reviews_google.drop("time")

# Lista de columnas a eliminar
columnas_a_eliminar = ['pics', 'resp']

# Elimina las columnas especificadas
reviews_google = reviews_google.select([columna for columna in reviews_google.columns if columna not in columnas_a_eliminar])

# Obtener una lista de gmap_id únicos
gmap_id_unicos = metadata_sitios.select("gmap_id").distinct().collect()

# Extraer los valores únicos de la lista
gmap_id_unicos = [row["gmap_id"] for row in gmap_id_unicos]

#Toma las columnas que pertenecian originalmente a reviews_google

reviews_google = reviews_google.filter(col("gmap_id").isin(gmap_id_unicos))



# Inicializa el analizador de sentimientos Vader
analyzer = SentimentIntensityAnalyzer()

# Define una función UDF para aplicar el análisis de sentimientos y asignar los valores 0, 1 o 2
def vader_sentiment_analysis(text,rating):
     # Verifica si el texto no es None o vacío
    if text and isinstance(text, str):
        compound_score = analyzer.polarity_scores(text)['compound']
        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score > -0.05 and compound_score < 0.05:
            return 1  # Neutral
        else:
            return 0  # Negativo
    else:
        # Verifica el valor de la columna 'rating'
        if rating >= 4.0:
            return 2
        elif rating < 3.0:
            return 0
        else:

            return 1

# Registrar la función UDF
vader_sentiment_analysis_udf = udf(vader_sentiment_analysis, StringType())

reviews_google = reviews_google.withColumn('sentiment', vader_sentiment_analysis_udf(reviews_google['text'], reviews_google['rating']))



In [12]:
# Elimina duplicados y seleccionar las columnas 'user_id' y 'name'
user_google = reviews_google.select('user_id', 'name').dropDuplicates()

# Elimina la columna 'name' del DataFrame original en PySpark
reviews_google = reviews_google.drop('name')

In [13]:
#Se lee el archivo business de yelp

with open("/content/drive/My Drive/Yelp/business.pkl", 'rb') as archivo:

        business_yelp = pickle.load(archivo)


In [14]:
#Elimina las columnas repetidas que tienen casi todos sus datos nulos
business_yelp=business_yelp.iloc[:,0:14]

# Lista de columnas a eliminar
columnas_a_eliminar = ['hours', 'attributes', 'is_open']

# Elimina las columnas especificadas
business_yelp = business_yelp.drop(columnas_a_eliminar, axis=1)

# Cambiar el nombre de la columna 'review_count' a 'num_of_reviews'
business_yelp= business_yelp.rename(columns={'review_count': 'num_of_reviews'})

# Filtra las filas donde la columna 'categories' contiene la cadena 'Restaurants'
business_yelp = business_yelp[business_yelp['categories'].str.contains('Restaurants', case=False, na=False)]

#Filtra los datos de los tres estados que necesitamos
business_yelp=business_yelp[(business_yelp['state']=='NV') | ((business_yelp['state']=='FL') | (business_yelp['state']=='VA'))]

business_yelp.reset_index(inplace=True,drop=True)



In [17]:
#Se lee el archivo review de yelp

review_yelp = spark.read.json("/content/drive/My Drive/Yelp/review.json")

In [18]:
business_ids= business_yelp['business_id'].tolist()

#Cambia el nombre de la columna 'stars' a 'rating'

review_yelp = review_yelp.withColumnRenamed("stars", "rating")

#Hace analisis de sentimientos

review_yelp = review_yelp.withColumn('sentiment', vader_sentiment_analysis_udf(review_yelp['text'],review_yelp['rating']))

# Filtrar las filas en review_yelp donde 'business_id' está en la lista
review_yelp = review_yelp.filter(col('business_id').isin(business_ids))

# Convierte la columna 'date' a tipo timestamp
review_yelp = review_yelp.withColumn("date", col("date").cast("timestamp"))

# Crear las columnas 'year', 'month' y 'day'
review_yelp = review_yelp.withColumn("year", year("date"))
review_yelp = review_yelp.withColumn("month", month("date"))
review_yelp = review_yelp.withColumn("day", dayofmonth("date"))

# Eliminar la columna original 'date'
review_yelp = review_yelp.drop("date")

#Toma las columnas que necesito

reviews_yelp=user_google = review_yelp.select('review_id','business_id','user_id', 'rating','year','month','day','text','sentiment')

In [19]:
#Se lee el archivo tip de yelp

tip_yelp=[]

with open("/content/drive/My Drive/Yelp/tip.json","r", encoding= 'utf-8') as filejson:


                # Procesa cada línea del archivo como un objeto JSON
                for linea in filejson:

                        # Intenta cargar la línea como un objeto JSON
                        objeto = json.loads(linea)

                        tip_yelp.append(objeto)

tip_yelp=pd.DataFrame(tip_yelp)


In [20]:
# Filtrar las filas basadas en 'business_ids'
tip_yelp= tip_yelp[tip_yelp['business_id'].isin(business_ids)]

# Convertir la columna 'date' a tipo datetime
tip_yelp['date'] = pd.to_datetime(tip_yelp['date'])

# Crear las columnas 'year', 'month' y 'day'
tip_yelp['year'] = tip_yelp['date'].dt.year
tip_yelp['month'] = tip_yelp['date'].dt.month
tip_yelp['day'] = tip_yelp['date'].dt.day

# Eliminar la columna original 'date'
tip_yelp = tip_yelp.drop(columns=['date','compliment_count'])

tip_yelp.reset_index(inplace=True,drop=True)

<ipython-input-20-7c70b2105ad0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tip_yelp['date'] = pd.to_datetime(tip_yelp['date'])


In [21]:
#Se lee el archivo user de yelp

users_yelp= pd.read_parquet("/content/drive/My Drive/Yelp/user.parquet")

In [22]:
# Selecciona las columnas 'user_id' y 'name'
users_yelp = users_yelp[['user_id', 'name']]